In [1]:
# ! pip install wildlife-tools
# ! pip install wildlife-datasets
# ! git clone https://github.com/WildlifeDatasets/wildlife-tools

In [2]:
import matplotlib.pyplot as plt
import torch
import pandas as pd
import torchvision.transforms as T
import timm

from wildlife_datasets import datasets
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
# dataset = datasets.MPDD('data\\MPDD')

root = 'data\\MPDD'
datasets.MPDD.get_data(root)

DATASET MPDD: DOWNLOADING STARTED.


v5j6m8dzhv-1.zip: 29.6MB [00:20, 1.43MB/s]                              


DATASET MPDD: EXTRACTING STARTED.
DATASET MPDD: FINISHED.



In [4]:
d = datasets.FriesianCattle2015(root)
dataset = WildlifeDataset(d.df, d.root)

This dataset is outdated. You may want to call a newer version such as FriesianCattle2015v2.


In [5]:
model_name = 'hf-hub:BVRA/MegaDescriptor-T-224'
model = timm.create_model(model_name, num_classes=0, pretrained=True)
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
extractor = DeepFeatures(model)

In [6]:
df1 = dataset.metadata[dataset.metadata['identity'] == 1]
df2 = dataset.metadata[dataset.metadata['identity'] == 2]
df_database = pd.concat((df1.iloc[2:], df2.iloc[2:]))
df_query = pd.concat((df1.iloc[:2], df2.iloc[:2]))

In [7]:
d_database = WildlifeDataset(df_database, d.root, transform=transform, img_load='crop_black')
d_query = WildlifeDataset(df_query, d.root, transform=transform, img_load='crop_black')

In [8]:
database = extractor(d_database)
query = extractor(d_query)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.64s/it]


In [9]:
df_database.size, df_query.size

(144, 16)

In [10]:
# from wildlife_tools.inference import KnnMatcher

# matcher = KnnMatcher(database)
# matcher_output =[]
# for i in outputs:
#     matcher_output.append(matcher([i]))


similarity_function = CosineSimilarity()
similarity = similarity_function(query, database)['cosine']
classifier = KnnClassifier(k=1, database_labels=d_database.labels_string)
predictions = classifier(similarity)


print(len(predictions))

4


c:\Python312\Lib\site-packages\wildlife_tools\inference\classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [11]:
predictions

array(['1', '1', '2', '2'], dtype=object)

In [12]:
len(set(dataset.metadata.identity))

40

In [18]:
from PIL import Image

def extract_features(image_path):
    img = Image.open(r'data\FriesianCattle2015\\'+image_path)
    img_tensor = transform(img).unsqueeze(0)  # Transform image and add batch dimension
    with torch.no_grad():
        features = model(img_tensor)  # Extract features using the pre-trained model
    return features


In [19]:
df1 = dataset.metadata[dataset.metadata['identity'] == 1]
df2 = dataset.metadata[dataset.metadata['identity'] == 2]
df_concat = pd.concat([df1, df2])

# Extract features for database and query images
database_features = torch.cat([extract_features(row['path']) for _, row in df_concat.iloc[2:].iterrows()])
query_features = torch.cat([extract_features(row['path']) for _, row in df_concat.iloc[:2].iterrows()])


# Similarity calculation (cosine similarity)
similarity = torch.nn.functional.cosine_similarity(query_features, database_features, dim=1)

# Classification (assigning labels based on highest similarity)
predictions = torch.argmax(similarity, dim=1)

print(predictions.tolist())  # Print the predicted labels


RuntimeError: The size of tensor a (2) must match the size of tensor b (38) at non-singleton dimension 0

In [21]:
similarity.size

144